In [ ]:
from skimage.io import imread
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, Add
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
import keras.optimizers as optimizers
from keras.applications.vgg16 import VGG16
#dimension of target image
IMG_HEIGHT = 576
IMG_WIDTH = 720
BATCH_SIZE = 4

#DATA GENERATION
def load_img(path):
    img = imread(path)
    img = img/ 255
    return img

def img_generator(img_list,batch_size):
    while True:
        X_batch_input = []
        y_batch_input = []
        degraded_path =  '/home/romain/datasets/dataset_clean_degraded/degraded/'
        clean_path =  '/home/romain/datasets/dataset_clean_degraded/clean/'
        batch_path = np.random.choice(a = img_list, size= batch_size)
        for img_name in batch_path:
            current_X = load_img(degraded_path+img_name)
            current_y = load_img(clean_path+img_name)
            X_batch_input += [ current_X ]
            y_batch_input += [ current_y ]
        batch_X = np.array(X_batch_input)
        batch_y = np.array(y_batch_input)
        yield (batch_X, batch_y)

#MODEL DEFINITION
model = Sequential()
init = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
layer_dict = dict([(layer.name, layer) for layer in base_model.layers])
init_1 = layer_dict['block3_pool'].output
x = Convolution2D(8, (3, 3), activation='relu', padding='same')(init_1)
x = UpSampling2D((2, 2))(x)

# DeConv2
x = Convolution2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)

# Deconv3
x = Convolution2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
level1_1 = Convolution2D(16, (3, 3), activation='relu', padding='same')(x)
level2_1 = Convolution2D(16, (3, 3), activation='relu', padding='same')(level1_1)
level2_2 = Conv2DTranspose(16, (3, 3), activation='relu', padding='same')(level2_1)
level2 = Add()([level2_1, level2_2])
level1_2 = Conv2DTranspose(16, (3, 3), activation='relu', padding='same')(level2)
level1 = Add()([level1_1, level1_2])
decoded = Convolution2D(3, (5, 5), activation='linear', padding='same')(level1)
model = Model(base_model.input, decoded)
adam = optimizers.Adam(lr=1e-3)
model.compile(optimizer=adam, loss='mse', metrics=['mean_squared_error'])
model.summary()

#TRAINING
img_list = os.listdir('/home/romain/datasets/dataset_clean_degraded/degraded/')
gen = img_generator(img_list, BATCH_SIZE)

STEPS_PER_EPOCH = np.ceil(5500/BATCH_SIZE)
history = model.fit_generator(gen, steps_per_epoch=STEPS_PER_EPOCH, epochs=2)

In [ ]:
import pickle
pickle.dump( model , open( "model-save-pickle-PreESRCNN.p", "wb"))
pickle.dump( history , open( "history-save-pickle-PreESRCNN.p", "wb"))

In [9]:
import pickle
model = pickle.load( open("C:/Users/Mathieu/CESI/Pickle/save-pickle-autoencoder.p", "rb"))

In [10]:
from keras.utils import plot_model
plot_model(model, to_file='model-autoencoder.png')
